In [ ]:
# Install required packages
!pip install -q -U openai-whisper
!pip install -q --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install python-docx

# Import necessary libraries
import whisper
import torch
import yt_dlp as youtube_dl
from pathlib import Path
from whisper.utils import get_writer
from docx import Document
from google.colab import drive
drive.mount('/content/gdrive')

# Global Variables
Model_Type = "large"
input_directory = "/content/gdrive/My Drive/Upload/"
output_directory = "/content/gdrive/My Drive/Output/"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def process_input(input_data):
    if input_data.startswith("https://www.youtube.com/"):
        # Process YouTube URL
        ydl_opts = {
            'format': 'bestaudio/best',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
            'outtmpl': '%(title)s.%(ext)s'
        }

        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(input_data, download=False)
            ydl.download([input_data])

        data_file = info_dict['title'] + '.mp3'
    else:
        # Process audio file
        data_file = input_directory + input_data

    return data_file

def transcribe_and_save(data_file):
    # Transcribe audio and get the result
    model = whisper.load_model(Model_Type, device=DEVICE)
    result = model.transcribe(data_file, language='zh')

    # Save Word document
    text_file_name = Path(data_file).stem
    doc = Document()
    doc.add_heading(text_file_name, 0)
    doc.add_paragraph(result["text"])
    doc_filename = f"{text_file_name}.docx"
    doc.save(Path(output_directory) / doc_filename)

# Input file name or youtube link
user_input = input("Enter YouTube URL or file name: ")
processed_file = process_input(user_input)
transcribe_and_save(processed_file)